In [81]:
# my practice code from https://www.youtube.com/playlist?list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S
# Sarcasm Dataset from kaggle: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection
# Citation:
    # 1. Misra, Rishabh and Prahal Arora. "Sarcasm Detection using News Headlines Dataset." AI Open (2023).
    # 2. Misra, Rishabh and Jigyasa Grover. "Sculpting Data for ML: The first act of Machine Learning." ISBN 9798585463570 (2021).
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import json

In [82]:
# import data
# with open('Sarcasm_Headlines_Dataset.json', 'r') as f:
#     datastore = json.loads(f)
    
def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

data = list(parse_data('./Sarcasm_Headlines_Dataset.json'))

In [83]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [84]:
# getting data ready for Neural Network
sentences = []
labels = []
urls = []

for item in data:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

# splitting into training and test sets
training_sentences = sentences[0 : training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0 : training_size]
testing_labels = labels[training_size:]

# Tokenization
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok) # including out of vocabulary words
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
# print(word_index)

# Sequencing
training_sequences = tokenizer.texts_to_sequences(training_sentences)
# print(sequences)

# Padding
training_padded = pad_sequences(training_sequences, maxlen = max_length,
                                padding = padding_type, truncating = trunc_type) # to make the sequences the same size
print(training_padded[0])
print(training_padded.shape)

[ 328    1  799 3405 2404   47  389 2214    1    6 2614 8863    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
(20000, 100)


In [85]:
# Testing sets, ready for evaluation
# Sequencing
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
# Padding
testing_padded = pad_sequences(testing_sequences, maxlen = max_length,
                                padding = padding_type, truncating = trunc_type)

print(testing_padded.shape)
print(len(testing_labels))

(6709, 100)
6709


In [86]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [87]:
# Building the Neural Network
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [88]:
# Compiling the model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [89]:
# Fitting the model
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs = num_epochs,
                    validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 3s - loss: 0.6705 - accuracy: 0.5791 - val_loss: 0.6059 - val_accuracy: 0.7782 - 3s/epoch - 6ms/step
Epoch 2/30
625/625 - 2s - loss: 0.4479 - accuracy: 0.8301 - val_loss: 0.3928 - val_accuracy: 0.8354 - 2s/epoch - 4ms/step
Epoch 3/30
625/625 - 2s - loss: 0.3176 - accuracy: 0.8740 - val_loss: 0.3571 - val_accuracy: 0.8495 - 2s/epoch - 4ms/step
Epoch 4/30
625/625 - 2s - loss: 0.2651 - accuracy: 0.8971 - val_loss: 0.3431 - val_accuracy: 0.8563 - 2s/epoch - 4ms/step
Epoch 5/30
625/625 - 2s - loss: 0.2296 - accuracy: 0.9128 - val_loss: 0.3417 - val_accuracy: 0.8551 - 2s/epoch - 4ms/step
Epoch 6/30
625/625 - 2s - loss: 0.2042 - accuracy: 0.9216 - val_loss: 0.3606 - val_accuracy: 0.8493 - 2s/epoch - 4ms/step
Epoch 7/30
625/625 - 2s - loss: 0.1800 - accuracy: 0.9323 - val_loss: 0.3665 - val_accuracy: 0.8483 - 2s/epoch - 4ms/step
Epoch 8/30
625/625 - 2s - loss: 0.1630 - accuracy: 0.9388 - val_loss: 0.3678 - val_accuracy: 0.8550 - 2s/epoch - 4ms/step
Epoch 9/30
625/625 - 2s 

In [93]:
# Predicting
sentence = ['welcome to real life. It sucks, you are gonna love it!']

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen = max_length,
                                padding = padding_type, truncating = trunc_type)
print(model.predict(padded))

1/1 [==============================] - 0s 24ms/step
[[0.9680223]]
